<a href="https://colab.research.google.com/github/tahe02/BBO-LP/blob/main/batch_blackbox_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing
from math import erfc
from copy import deepcopy
from scipy.stats import qmc

RANDOM_STATE = 0

np.random.seed(RANDOM_STATE)

Synthetic Functions

In [2]:
class synthetic_function():
    def __init__(self, d):
        self.SobolSampler = qmc.Sobol(d=d, scramble=False)

class Ackley(synthetic_function):
    def __init__(self, d):
        super().__init__(d)
        self.dim = d
        self.lower_bounds = [-32.768] * d 
        self.upper_bounds = [32.768] * d 
        self.candidates = self.SobolSampler.random_base2(m=14) 

    def f(self, x):
        a = 20.0
        b = 0.2
        c = 2.0*np.pi
        sum1 = np.sum(x**2, axis=1)
        sum2 = np.sum(np.cos(c*x), axis=1)
        res = -a * np.exp(-b*np.sqrt((1/self.dim) * sum1)) - np.exp((1/self.dim) * sum2) + a + np.exp(1)
        return -res

class Schwefel(synthetic_function):
    def __init__(self, d):
        super().__init__(d)
        self.dim = d
        self.lower_bounds = [-500.0] * d 
        self.upper_bounds = [500.0] * d
        self.candidates = self.SobolSampler.random_base2(m=14)

    def f(self, x):
        res = 418.9829*self.dim - np.sum(x*np.sin(np.sqrt(np.abs(x))),axis=1)
        return res

class Rastrigin(synthetic_function):
    def __init__(self, d):
        super().__init__(d)
        self.dim = d
        self.lower_bounds = [-5.12] * d 
        self.upper_bounds = [5.12] * d
        self.candidates = self.SobolSampler.random_base2(m=14)

    def f(self, x):
        tmp = x**2 - 10.0*np.cos(2*np.pi*x)
        res = 10.0*self.dim - np.sum(tmp,axis=1)
        return res

Defining functions

In [3]:
# Defining acquisition functions

def alpha_ei(mean_s, std_s, cur_best, epsilon=0.01):
    meanY = mean_s
    stdY = std_s
    z = (meanY - cur_best - epsilon) / stdY
    res = (meanY - cur_best - epsilon) * norm.cdf(z) + stdY * norm.pdf(z)
    return res

def alpha_ucb(mean_s, std_s, cur_best, beta=1.98):
    res = mean_s+ beta*std_s
    return res


# Creating surrogate model classes

class GPsur():
  def __init__(self):
      self.model = GaussianProcessRegressor(kernel=ConstantKernel()*Matern(), n_restarts_optimizer=10, random_state=RANDOM_STATE)
  
  def fit(self, x, y):
    self.model.fit(x,y)

  def predict(self, x, return_std=True):
    return self.model.predict(x, return_std=return_std)

class RFsur():
  def __init__(self):
      self.model = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
  
  def fit(self, x, y):
    self.model.fit(x,y)

  def predict(self, x, return_std=True):
    preds = []
    for tree in self.model.estimators_:
      preds.append(tree.predict(x))
    means = np.mean(preds, axis=0)
    stds = np.std(preds, axis=0)
    if return_std:
      return means, stds
    return means


# Estimating L-Lipschitz constant via maximum observed gradient


def compute_norm(x1, x2):

  norm = 0
  try:
    for i in range(len(x1)):
      norm += (x1[i] - x2[i])**2
  except TypeError:
    norm = (x1 - x2)**2

  return np.sqrt(norm)


def estimate_L_UB(x_obs, y_obs, current_L, bsize, iteration):

  rb = iteration*bsize
  for i in range(len(x_obs)):
    for j in range(rb, len(y_obs)):
      rise = y_obs[i] - y_obs[j]

      # Compute the distance between the two points
      dist = compute_norm(x_obs[i], x_obs[j])

      # Avoids div by 0 when points are the same
      if dist != 0:
        grad_norm = abs(rise/dist)
      else:
        grad_norm = -np.inf

      current_L = max(current_L, grad_norm)
  
  return current_L

# Acquisition transformation function
def g_transform(z):
  # if z >= 0:
  #   return z
  return np.log(1 + np.exp(z))


# Defining penalising function


def z_func(x_batch, sd, mu, L, M, pointspace):

  norm = [compute_norm(x_batch, p) for p in pointspace]

  z = [
      ((L * d)-M+mu)/np.sqrt(2 * sd**2) for d in norm
  ]
  return z



def phi(x_batch, sd, mu, L, M, pointspace):
  z = z_func(x_batch, sd, mu, L, M, pointspace)
  return [(erfc(-1*p))/2 for p in z]

def create_space(dimpoints, dim=1):
  
  if dim == len(dimpoints):
    return dimpoints[dim-1].reshape(-1,1)

  lower = create_space(dimpoints, dim+1)
  space = np.empty(0)
  for p1 in lower:
    for p2 in dimpoints[dim-1]:
      space = np.append(space, np.insert(p1, 0, p2))

  return space.reshape(-1, len(dimpoints)-dim+1)

def BBO_LP(x_obs, y_obs, nb, m, model, function, dimension, resolution, pointspace):

  maxes = []
  batch_variance = []
  Ls = []

  # Set L to -inf when no estimate exists
  L = -np.inf

  for t in range(m):
    # Define batch
    batch = []

    # Fit model to current dataset
    model.fit(x_obs.reshape(-1, dimension), y_obs)

    # Build acquisition function based on current fit   

    pred_means, pred_stds = model.predict(pointspace, return_std=True)
    alpha = alpha_ei(pred_means, pred_stds, np.max(y_obs))


    # Apply g-stransformation to acquisition function
    alpha_t_0 = [g_transform(p) for p in alpha]
    
    # Estimate L
    L = estimate_L_UB(x_obs.reshape(-1, dimension), y_obs, L, nb, t)
    Ls.append(L)

    # Estimate M
    M = max(y_obs)
    maxes.append(M)

    # Visualising Acquisition
    # zscore = 1.96

    # pred_uppers = pred_means+pred_stds*zscore # the 97.5th percentile of predictions
    # pred_lowers = pred_means-pred_stds*zscore # the 2.5th percentile of predictions

    # max_alpha_x = pointspace[np.argmax(alpha)]
    # max_alpha_y = np.max(alpha)

    # max_g0_x = pointspace[np.argmax(alpha_t_0)]
    # max_g0_y = np.max(alpha_t_0)

    # fig, ax1 = plt.subplots(figsize=(10,5))
    # ax1.plot(pointspace.flatten(),function.f(pointspace), color="black")
    # ax1.fill_between(pointspace.flatten(), pred_uppers, pred_lowers,color="royalblue", alpha=0.3)
    # ax1.plot(pointspace.flatten(),pred_means, color="royalblue")
    # ax1.scatter(x_obs, y_obs)


    # ax1.set_xlabel("x")
    # ax1.set_ylabel("f(x)")

    # ax2 = ax1.twinx()
    # ax2.plot(pointspace.flatten(),alpha, color="green")
    # ax2.fill_between(pointspace.flatten(), 0, alpha, color="springgreen",alpha=0.3, label=r"$\~{\alpha}_{ei}(x)$")
    # ax2.scatter(max_alpha_x, max_alpha_y, s=200, marker="*", color="yellow", edgecolor="black", zorder=10)
    # ax2.set_ylim(0.0, np.max(alpha)*4.0)

    # plt.title('Before Batch Selection {}'.format(t+1))
    # plt.legend()
    # plt.savefig(f'batch_{t+1}_acq_RF.png')
    # plt.show()

    alpha_t_j = deepcopy(alpha_t_0)

    for j in range(nb):
      # M step
      batch_elem = pointspace[np.argmax(alpha_t_j)]      
      batch.append(batch_elem)

      # P step
      phi_mult = [1 for p in pointspace.reshape(-1, dimension)]
 
      sd = pred_stds[pointspace.tolist().index(batch_elem.tolist())]
      mu = pred_means[pointspace.tolist().index(batch_elem.tolist())]

      for k in range(j+1):
        new_phi = phi(batch[k], sd, mu, L, M, pointspace)
        phi_mult = [phi_mult[i] * new_phi[i] for i in range(len(phi_mult))]
      
      
      alpha_t_j = [alpha_t_0[i] * phi_mult[i] for i in range(len(alpha_t_0))]

      # Visualising Penalisation
      # zscore = 1.96

      # pred_uppers = pred_means+pred_stds*zscore # the 97.5th percentile of predictions
      # pred_lowers = pred_means-pred_stds*zscore # the 2.5th percentile of predictions

      # max_g_x = pointspace[np.argmax(alpha_t_j)]
      # max_g_y = alpha_t_j[np.argmax(alpha_t_j)]
      
      # fig, ax1 = plt.subplots(figsize=(10,5))
      # ax1.plot(pointspace.flatten(),function.f(pointspace), color="black")
      # ax1.fill_between(pointspace.flatten(), pred_uppers, pred_lowers,color="royalblue", alpha=0.3)
      # ax1.plot(pointspace.flatten(),pred_means, color="royalblue")
      # ax1.scatter(x_obs, y_obs)


      # ax1.set_xlabel("x")
      # ax1.set_ylabel("f(x)")

      # ax2 = ax1.twinx()

      # ax2.plot(pointspace.flatten(),alpha_t_j, color="orange")
      # ax2.fill_between(pointspace.flatten(), 0, alpha_t_j, color="orange",alpha=0.3, label=r"g($\~{\alpha}_{ei}(x)$)")
      # ax2.scatter(max_g_x, max_g_y, s=200, marker="*", color="yellow", edgecolor="black", zorder=10)   

      # ax2.plot(pointspace.flatten(), phi_mult, color='purple')

      # ax2.set_ylim(0, np.max(alpha_t_j)*4)

      # plt.title('transformed g after {} batch selections for batch {} ({}) with L = {}'.format(j+1, t+1, batch, L))

      # plt.savefig(f'batch_{t+1}_selection{j+1}_RF.png')
      # plt.show()
    
    # Evaluate the variance of points in the batch
    batch_variance.append(np.var(batch))

    # Evaluate the function at all batch point locations
    for i in range(len(batch)):
      x_obs = np.append(x_obs, batch[i])
      y_obs = np.append(y_obs, function.f(batch[i].reshape(-1,dimension)))
 
  model.fit(x_obs.reshape(-1, dimension), y_obs)
  pred_means, pred_stds = model.predict(pointspace, return_std=True)

  # Visualising Final Fit
  # zscore = 1.96

  # pred_uppers = pred_means+pred_stds*zscore # the 97.5th percentile of predictions
  # pred_lowers = pred_means-pred_stds*zscore # the 2.5th percentile of predictions

  # fig, ax1 = plt.subplots(figsize=(10,5))
  # ax1.plot(pointspace.flatten(),function.f(pointspace), color="black")
  # ax1.fill_between(pointspace.flatten(), pred_uppers, pred_lowers,color="royalblue", alpha=0.3)
  # ax1.plot(pointspace.flatten(),pred_means, color="royalblue")
  # ax1.scatter(x_obs, y_obs)


  # ax1.set_xlabel("x")
  # ax1.set_ylabel("f(x)")

  # plt.title('Final Fit')
  # plt.legend()
  # plt.savefig('final_RF.png')
  # plt.show()

  return x_obs.reshape(-1, dimension)[np.argmax(y_obs)]

Experiment parameters

In [4]:
dim = 1
bounds = [[-10,10]]*dim
func = Ackley(dim)
bsize = 3
iterations = 10
resolution = 20

Create search space

In [5]:
dimpoints = []

for b in bounds:
  dimpoints.append(np.linspace(b[0], b[1], (b[1] - b[0])*resolution))

pointspace = create_space(dimpoints)

Run experiment

In [7]:
# Generating intitial points
x_obs = np.random.uniform(low=-10, high=10, size=(bsize,dim))
y_obs = func.f(x_obs.reshape(-1, dim))

BBO_LP(
    x_obs, y_obs, nb=bsize, m=iterations, model=RFsur(), function=func, dimension=dim, resolution=resolution, pointspace=pointspace
)

array([-0.02506266])